# 02_DataUnderstanding_Preparation
## Sentiment Analysis for MySejahtera Apps
### Group members : Queenie, Deva, Ethan Ong, Lawrence, Oscar

## Content
- [Importing Libraries](#Importing-libraries)
- [Data Collection](#Data-Collection)
- [Data Cleaning and Preprocessing](#Data-Cleaning-and-preprocessing)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
# from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from google_play_scraper import Sort, reviews

%matplotlib inline

# Data Collection

Users can rate and write reviews for MySejahtera's app on GooglePlay with a star rating and thumbs-up system. The ratings are on a 5-point scale, with 1 being the lowest rate score and 5 being the highest rate score. The goal of the project is to predict the review written is classified as positive or negative sentiment based on textual data. Our group will scrape real user reviews on GooglePlay using GooglePlay Scraper. 

Google-Play-Scraper provides an API to crawl through Google Play. We used pip install google-play-scraper to install the package and scraped users' reviews and rating scores on MySejahtera's app.

The reviews were collected in batches, according to their scores (1-5). This was done in an attempt to achieve a balanced dataset with roughly the same number of reviews for each score. Also, in order to gather reviews that had more text and were written recently, we set up the google play scraper to scrape from both review types, 'Most relevant' and 'Newest'.

#### Pre-installation for GooglePlay Scraper

_Data Collection part will be commented out to prevent accidentally generate a second set of data._

In [2]:
#pip install google-play-scraper

In [3]:
# app_pick = {
#     "mysj": ("my.gov.onegovappstore.mysejahtera", "mysj_android_reviews.csv", "mysj_android_reviews_parsed.csv"),
# }

# android_app_package, file_reviews_csv, file_reviews_parsed_csv = app_pick["mysj"]
# print(f"Pick {android_app_package}")

In [4]:
# # Function to scrape reviews on google play store
# # app: the url of the app we want to scrape
# # score: number of stars rated by users
# # n_loops: the number of loops to collect reviews in batches of 200

# def reviews_scraper(app, score, n_loops):
#      app_reviews_tmp = []
#      for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]: # Collect both reviews types - 'most relevant' and 'newest'
#          for i in range(n_loops):
#              rvs, continuation_token = reviews(app,
#                                                lang='en',
#                                                country='my',
#                                                sort=sort_order,
#                                                count=200, # 200 is the maximum number of reviews per page supported by Google Play
#                                                filter_score_with=score,
#                                                continuation_token=None if i==0 else continuation_token) # To begin crawling from where it last left off
#              for r in rvs:
#                  r['sort_order'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
#                  r['app_id'] = app
#              app_reviews_tmp.extend(rvs)
#              print(f'No. of reviews collected (score {score}): ' + str(len(rvs)))
#      return app_reviews_tmp

In [5]:
# rev_score1 = reviews_scraper(app=android_app_package, score=1, n_loops=5)
# rev_score2 = reviews_scraper(app=android_app_package, score=2, n_loops=5)
# rev_score3 = reviews_scraper(app=android_app_package, score=3, n_loops=5)
# rev_score4 = reviews_scraper(app=android_app_package, score=4, n_loops=5)
# rev_score5 = reviews_scraper(app=android_app_package, score=5, n_loops=5)
# app_reviews = rev_score1 + rev_score2 + rev_score3 + rev_score4 + rev_score5
# print(f"Collected records: {len(app_reviews)}")

In [6]:
# pd.DataFrame(app_reviews).to_csv(f'C:/Users/eoa/Downloads/{file_reviews_csv}', index=False)


In [7]:
review_data = pd.read_csv('C:/Users/oscar/Desktop/UM/WQD7003 Data Analytics/mysj_android_reviews.csv', parse_dates = ['at', 'repliedAt'])


In [8]:
# To display the first 5 row of data
review_data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sort_order,app_id
0,ce335359-73d9-4fa6-a487-4c66cef04495,pao yin lee,https://play-lh.googleusercontent.com/a/ALm5wu...,Some buttons are unable to function. I am tire...,1,494,1.0.35,2021-07-06 20:32:00,"Dear User, Kindly force close your app and try...",2021-08-06 14:22:00,most_relevant,my.gov.onegovappstore.mysejahtera
1,81adb7e4-4b1a-467b-abb3-777e8f06bedc,Eleanor Chooi,https://play-lh.googleusercontent.com/a/ALm5wu...,After updating to the latest version which ena...,1,361,1.0.48,2021-10-21 15:38:00,"Hello, please give app permission to access fi...",2021-10-21 16:49:00,most_relevant,my.gov.onegovappstore.mysejahtera
2,3b5c2452-156b-45c3-b663-954052103ddd,Irene Lee,https://play-lh.googleusercontent.com/a/ALm5wu...,With the new tracking function automatically a...,1,201,1.1.4,2022-01-01 20:28:00,"Dear user, kindly go to device settings > apps...",2022-07-01 10:49:00,most_relevant,my.gov.onegovappstore.mysejahtera
3,2c9ed39c-ce1a-49d3-8310-6f093274078d,Nashrul Iqram,https://play-lh.googleusercontent.com/a-/ACNPE...,"This app is good, but the one that bugs me the...",1,40,1.0.39,2021-07-29 16:43:00,"Dear User, We are truly sorry for your unpleas...",2021-05-03 17:47:00,most_relevant,my.gov.onegovappstore.mysejahtera
4,e8ac5477-ae62-4607-933f-5ed9ac86a0ef,Teik Lee Teoh,https://play-lh.googleusercontent.com/a-/ACNPE...,The latest update for MySJ trace is just utter...,1,349,1.1.4,2022-04-01 09:04:00,"Dear User, We are truly sorry for your unpleas...",2022-11-01 12:12:00,most_relevant,my.gov.onegovappstore.mysejahtera


In [9]:
# 10,000 rows of data is extracted
review_data.shape

(10000, 12)

In [10]:
review_data.info

<bound method DataFrame.info of                                   reviewId                   userName  \
0     ce335359-73d9-4fa6-a487-4c66cef04495                pao yin lee   
1     81adb7e4-4b1a-467b-abb3-777e8f06bedc              Eleanor Chooi   
2     3b5c2452-156b-45c3-b663-954052103ddd                  Irene Lee   
3     2c9ed39c-ce1a-49d3-8310-6f093274078d              Nashrul Iqram   
4     e8ac5477-ae62-4607-933f-5ed9ac86a0ef              Teik Lee Teoh   
...                                    ...                        ...   
9995  62b6a1eb-bd20-417d-9f8c-5eb4ac0802ca                  Rose Wati   
9996  f3e5615a-a4d6-43cd-acca-fa779b939613      Elizabeth Regina Mary   
9997  5f4490b9-43ed-40c1-ad49-97f50c014446           Prakash Muniandy   
9998  e96475be-89ae-432c-9da3-11b35fb872a5                Fitri Rosli   
9999  57d31ff3-4ed3-45ce-a701-1792973fb463  Akmalhakim88 Akmalhakim88   

                                              userImage  \
0     https://play-lh.googleuser

In [11]:
# There is some null values in app versions and developer replies and one null value in user content
review_data.isnull().sum()

reviewId                   0
userName                   0
userImage                  0
content                    1
score                      0
thumbsUpCount              0
reviewCreatedVersion    1092
at                         0
replyContent            3718
repliedAt               3718
sort_order                 0
app_id                     0
dtype: int64

In [12]:
# Check how many records are retrieved from each type of rating score.
review_data['score'].value_counts().sort_index()

1    2000
2    2000
3    2000
4    2000
5    2000
Name: score, dtype: int64

# Data Cleaning and preprocessing

## Data Cleaning

In [13]:
#811 duplicated row of data
review_data[review_data.duplicated(['userName', 'content', 'at'])].shape

# Check the null value in content data which is row 7922
review_data.loc[review_data["content"].isnull()]

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sort_order,app_id
7922,ecde6b48-91f4-496c-9c29-177a7e97d8ac,Azhar Mat daud,https://play-lh.googleusercontent.com/a-/ACNPE...,NaN,4,0,1.1.4,2021-12-30 10:40:00,NaN,NaT,newest,my.gov.onegovappstore.mysejahtera


In [14]:
#drop duplicated data

review_data.drop_duplicates(['userName', 'content', 'at'], inplace=True)

#drop row 7922 data which contains null value in content column
review_data.drop(7922,inplace=True)

In [15]:
# Reindex the dataframe
review_data.reset_index(drop=True, inplace=True)

#9189 rows of data left after dropping duplicated
review_data.shape

(9188, 12)

In [16]:
# Check how many records we have for each rating score after dropping duplicated data
review_data['score'].value_counts().sort_index()


1    1835
2    1721
3    1812
4    1876
5    1944
Name: score, dtype: int64

In [17]:
# Renaming the columns to lowercase and use underscore
review_data.rename(columns={'reviewId': 'review_id', 
                        'userName': 'username', 
                        'userImage': 'user_image', 
                        'thumbsUpCount': 'thumbs_up_count', 
                        'reviewCreatedVersion': 'review_created_version', 
                        'replyContent': 'reply_content',
                        'repliedAt': 'replied_at'},
              inplace=True)

In [18]:
#create a second set of data for experiment 2
review_data2 = review_data.copy()

In [19]:
#experiment 1 data set with positive 0 index 5<x<3, 1 index 2<x<1
review_data['target'] = review_data['score'].map(lambda x: 1 if x < 3 else 0)

In [20]:
#experiment 2 dataset with positive 0 index 5<x<4, 1 index 3<x<1
review_data2['target'] = review_data['score'].map(lambda x: 1 if x < 4 else 0)

In [21]:
#dataset experiment 1
review_data['target'].value_counts()


0    5632
1    3556
Name: target, dtype: int64

In [22]:
#dataset experiment 2
review_data2['target'].value_counts()

1    5368
0    3820
Name: target, dtype: int64

## Data Preprocessing

In [23]:
def stem_text(raw_text):
  
    # Remove HTML tags
    review_text = BeautifulSoup(raw_text).get_text()
    
    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    # Convert words to lower case and split each word up
    words = letters_only.lower().split()
    
    # Searching through a set is faster than searching through a list 
    # Hence, we will convert stopwords to a set
    stops = set(stopwords.words('english'))
    
    # Adding on stopwords that were appearing frequently in both positive and negative reviews 
    stops.update(['app','MySJ','Covid','Covid-19','MySejahtera','COVID-19','pandemic','Bluetooth','mysejahtera']) 
    
    # Remove stopwords
    meaningful_words = [w for w in words if w not in stops]
        
    # Instantiate PorterStemmer
    p_stemmer = PorterStemmer()
    
    # Stem words
    meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]        
   
    # Join words back into one string, with a space in between each word
    return(" ".join(meaningful_words))

In [24]:
#preprocess raw text dataset #1
review_data['content_stem'] = review_data['content'].map(stem_text)

# preprocessing raw text dataset #2
review_data2['content_stem'] = review_data2['content'].map(stem_text)

C:\Users\oscar\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\oscar\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [25]:
#This is the original text of the first review in our dataset
review_data.loc[0]['content']

'Some buttons are unable to function. I am tired in trying to reinstall the app again and again. It brings alot of inconveniences, please fix it immediately. For example: when I tried to press the button to check-in, the screen remains unchanged. I understand the government is swamped under the current condition of our country, but a poorly-functional app has the risk to discourage the people to use it. Thank you.'

In [26]:
#This is how the text looks like after stemming
review_data.loc[0]['content_stem']

'button unabl function tire tri reinstal bring alot inconveni pleas fix immedi exampl tri press button check screen remain unchang understand govern swamp current condit countri poorli function risk discourag peopl use thank'

In [27]:
review_data['content_clean_len'] = review_data['content_stem'].str.split().map(len)

review_data2['content_clean_len'] = review_data2['content_stem'].str.split().map(len)


In [28]:
#96 row of data with no meaningful words
review_data[review_data['content_clean_len']==0].shape


(96, 15)

In [29]:
#viewing data

review_data[review_data['content_clean_len']==0]['content']


1008        👍
1094      for
1250        🤮
1399      🥵🥵🥵
1488        👎
        ...  
8999    👍👍👍👍👍
9067        🤗
9088       👍🏻
9119    👍👍👍👍👍
9146     Here
Name: content, Length: 96, dtype: object

In [30]:
#dropping row without meaningful words

review_data = review_data.drop(review_data[review_data['content_clean_len']==0].index)

review_data2 = review_data2.drop(review_data2[review_data2['content_clean_len']==0].index)

#reset index
review_data.reset_index(drop=True, inplace=True)

review_data2.reset_index(drop=True, inplace=True)


In [31]:
# To check non-english/gliberrish data manually

# Load the list of non-english gibberish indexes from an excel file checked manually
df = pd.read_excel('C:/Users/oscar/Desktop/UM/WQD7003 Data Analytics/list_to_drop.xlsx',header=None)

# Select the column of numbers
column = df[0]

# Convert the column to a list of integers
review_to_drop = column.tolist()

# Print the resulting list
print(review_to_drop)

# Print the length of the list
print(len(review_to_drop))

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# #check the review that are non-english
review_data.loc[review_to_drop,'content']

In [ ]:
#drop first set of data
review_data.drop(review_to_drop,inplace=True)

#drop second set of data
review_data2.drop(review_to_drop,inplace=True)

In [ ]:
#To check the row of first dataset after dropping non-english/gibberish reviews
review_data.shape

In [ ]:
#To check the row of second dataset after dropping non-english/gibberish reviews
review_data2.shape

## Exploratory Data Analysis

We will perform EDA analysis to draw any insights from the dataset before it was proceed to the next phase which is Model Training.

_Please note that EDA part will solely using only dataset experiment 1 for analysis._
### Time frame of the review written

We will compare the distributions of the review dates across various ratings, 1-5.

In [ ]:
# Create histogram subplots for each rating (1-5)
# Each subplot represents the distribution of the dates at which the reviews were written
def subplot_histograms(main_title, list_of_titles, xlabels, list_of_colors):
    fig, ax = plt.subplots(5, 1, figsize=(8,10), sharex=True, sharey=True)
    for i in range(5):
        ax[i].hist(review_data[review_data['score']==(i+1)]['at'], bins=50, color=list_of_colors[i])
        ax[i].set_title(list_of_titles[i], weight='bold')
        ax[i].set_xlabel(xlabels)
        ax[i].set_ylabel('Frequency')
    fig.suptitle(main_title, fontsize=15, weight='bold')
    fig.tight_layout()
    fig.subplots_adjust(top=0.9)

In [ ]:
subplot_histograms(main_title="Distribution of Review Dates by Users' Ratings",
                   list_of_titles=['1-Star Rating','2-Star Rating','3-Star Rating','4-Star Rating','5-Star Rating'], 
                   xlabels='Date of Reviews',
                   list_of_colors = ['#B03A2E','#CB4335','#CD6155','#16A085','#1E8449'])

Based on the observation from the charts above, the distributions for each rating scores spread evenly across review dates that earliest dated back to January 2020 which is when the COVID-19 pandemic started in Malaysia. In an ideal world, we would like the reviews for each rating score to be collected in an equal quantities, over the same time period. By using the GooglePlay Scraper, despite that the reviews that we collected are not entirely in equal quantites but the gap from number of reviews between each rating score is small. In such, we are able to obtain a balanced class training set, consisting of a relatively equal split between positive and negative reviews. 

Another key observation that is worth to note here is the number of reviews for each rating score raised significantly between 2022 Jan and 2022 May. This may likely be a result of a newly introduced features in the apps which is MySJ Trace, another reason that can be identified is the arrival of the booster shot for vaccination.

In [ ]:
# 324 reviews written in the context of 'vaccine'
len(review_data[review_data['content'].str.contains('vaccine')])

In [ ]:
pd.set_option('display.max_colwidth', None)

# Reviews that mentioned the word 'vaccine'
review_data[review_data['content'].str.contains('vaccine')][['content','score']]

## Number of thumbs up received

In [ ]:
# Group by rating scores and find the average no. of thumbs up per score
review_data.groupby('score')['thumbs_up_count'].mean().plot(kind='bar', figsize=(8,5), color='skyblue')
plt.xlabel('Ratings', fontsize=12)
plt.xticks(rotation=0)
plt.ylabel('Number of thumbs up', fontsize=12)
plt.ylim(0,25)
plt.title('Average No. of Thumbs Up Received by Rating Scores', fontsize=15, weight='bold')
plt.tight_layout();

Negative reviews with 1 star rating received significantly more thumbs up on average when comparing to other star rating.

In [ ]:
# Show the top 5 reviews that received the highest number of thumbs up
review_data[['content','thumbs_up_count','score']].sort_values(by='thumbs_up_count', ascending=False).head()

## Number of meaningful words

We will compare the length of the reviews among positive vs negative sentiments.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,8), sharex=True, sharey=True)

# Plots the distribution of the length of negative reviews
neg_len = review_data[review_data['target']==1]['content_clean_len']
sns.distplot(neg_len, color='red', ax=ax[0])
ax[0].axvline(neg_len.mean(), linestyle='dashed', linewidth=3, c='#2980B9') # Plot the mean line
ax[0].text(x=neg_len.mean(),
           y=0.08,
           s=f'Mean: {round(neg_len.mean())}', 
           horizontalalignment='left', 
           verticalalignment='center',
           fontsize=13)
ax[0].set_title('No. of Meaningful Words in Negative Reviews', fontsize=15, fontweight='bold')
ax[0].set_xlabel('No. of meaningful words', size=12)

# Plots the distribution of the length of positive reviews
pos_len = review_data[review_data['target']==0]['content_clean_len']
sns.distplot(pos_len, color='green', ax=ax[1])
ax[1].axvline(pos_len.mean(), linestyle='dashed', linewidth=3, c='#2980B9') # Plot the mean line
ax[1].text(x=pos_len.mean(),
           y=0.08,
           s=f'Mean: {round(pos_len.mean())}', 
           horizontalalignment='left', 
           verticalalignment='center',
           fontsize=13)
ax[1].set_title('No. of Meaningful Words in Positive Reviews', fontsize=15, fontweight='bold')
ax[1].set_xlabel('No. of meaningful words', size=12)
plt.tight_layout();

Based on the observation from the charts above, both distribution are right skewed with only a small proportion of post that have >50 meaningful words. The average number of meaningful words in negative reviews (21 words) is higher than the number of meaningful words written in positive reviews (15 words). This can further implies that dissatisfied customer are more likely to write longer reviews.

In [ ]:
# Number of reviews that had >40 meaningful words
review_data[review_data['content_clean_len']>40]['target'].value_counts().sort_index()

In [ ]:
review_data['target'].value_counts()

In [ ]:
194/4662

Only about 4.16% of positive reviews have > 40 meaningful words.

In [ ]:
309/2874

About 10.8% of negative reviews have > 40 meaningful words.

## Wordclouds: Most frequently used words

Word clouds are useful in visualising some of the most frequently occurring words in positive and negative reviews.

In [ ]:
# Write a function to plot a wordcloud of the most frequently occurring words

def plot_wordcloud(target, title):
    
    # Clean text that we want to wordcloud. Target will be class 1 (negative reviews) or class 0 (positive reviews)
    text = review_data[review_data['target']==target]['content_stem'] 

    wordcloud = WordCloud(width=2000, 
                          height=1000, 
                          background_color='white', 
                          max_words=50, # Maximum no. of words in the wordcloud
                          stopwords=STOPWORDS).generate(' '.join(text)) # Remove stopwords
    
    plt.figure(figsize=(10,8))
    plt.title(title, fontsize=15, weight='bold')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off') # Removes the axis
    plt.tight_layout()

In [ ]:
# Wordcloud of the 50 most frequently occurring words among negative reviews
plot_wordcloud(target=1, title='Most Frequently Used Words among Negative Reviews (Ratings 1-3)')

The words 'updat' and 'check' are the most frequently seen words among reviews with negative sentiments.

In [ ]:
# Wordcloud of the 50 most frequently occurring words among positive reviews
plot_wordcloud(target=0, title='Most Frequently Used Words among Positive Reviews (Ratings 4-5)')

Although the words 'check' and 'update' are still on the top words seen in the reviews, words like 'thank', 'good' is showing up in positive reviews.

# Barplots

In [ ]:
# Plots top 20 uni-grams and bi-grams for positive and negative reviews

def barplot_cvec(target, titles, color, xlimit):
    
    words_series = review_data[review_data['target']==target]['content_stem']
    
    fig, ax = plt.subplots(1, 2, figsize=(25,12))
    
    ngram = [(1,1),(2,2)] # The ngrams that we would like to plot
    
    for i in range(2):
            
        # Use CountVectorizer to tokenize the text, 
        cvec = CountVectorizer(stop_words='english', ngram_range=ngram[i])

        # Save the tokens in a dataframe
        cvec_df = pd.DataFrame(cvec.fit_transform(words_series).todense(), columns=cvec.get_feature_names())
        sum_words = cvec_df.sum(axis=0) # Sum up the no. of occurences for each word
        top_words = sum_words.sort_values(ascending=False).head(20)
        top_words.sort_values(ascending=True).plot(kind='barh', color=color, ax=ax[i])

        # Adjust plot aesthetics
        ax[i].set_title(titles[i], size=25, weight='bold')
        ax[i].set_xlabel('Count', size=20)
        ax[i].set_xlim(xlimit) # Setting a limit so that the barplots are comparable
        ax[i].tick_params(axis='both', which='major', labelsize=20)
        ax[i].tick_params(axis='both', which='minor', labelsize=20)

    plt.tight_layout()

In [ ]:

# Plot the uni-grams and bi-grams for negative reviews
barplot_cvec(target=1,
             titles=['Top 20 Uni-grams among Negative Reviews','Top 20 Bi-grams among Negative Reviews'], 
             color='#CB4335',
             xlimit=(0,400))

__'Update', 'check' and 'vaccine'__ are the top 3 most frequently occurring uni-grams in negative reviews. __'please fix' and 'reset password' and 'many time'__ are the top 3 most frequently occurring bi-grams in negative reviews. We can thus infer that the users are facing problem with resetting their password frequently on the app.

In [ ]:
# Showing some of the negative reviews with the word 'update'
review_data[(review_data['target']==1) & (review_data['content_stem'].str.contains('updat'))][['content','score']].head()

In [ ]:
# Showing some of the negative reviews with the word 'vaccine'
review_data[(review_data['target']==1) & (review_data['content_stem'].str.contains('vaccin'))][['content','score']].head()

In [ ]:
# Plot the uni-grams and bi-grams for positive reviews
barplot_cvec(target=0,
             titles=['Top 20 Uni-grams among Positive Reviews','Top 20 Bi-grams among Positive Reviews'], 
             color='#239B56',
             xlimit=(0,800))

__'Check', 'Update', 'Use'__ are the top 3 most frequently seen uni-grams in positive reviews. The bi-grams also give us some context to the word __'qr code'__ and __'easi use'__, where it probably refers to an 'easy to use' apps by the users.

In [ ]:
# Showing some of the positive reviews with the word 'use'
review_data[(review_data['target']==0) & (review_data['content_stem'].str.contains('use'))][['content','score']].head()

In [ ]:
# Showing some of the positive reviews with the word 'check'
review_data[(review_data['target']==0) & (review_data['content_stem'].str.contains('check'))][['content','score']].head()

## Save both two set of dataset for modelling

In [ ]:
# export dataset out

review_data.to_csv('dataset experiment_1.csv')
review_data2.to_csv('dataset experiment_2.csv')